# Analysis: Store Price

Goal: Run linear regression on all features for target variable store-price.

In [1]:
import os
import pandas as pd
import numpy as np
import sklearn.model_selection as ms

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
%load_ext autoreload
%autoreload 2         # reload custom py file eveytime a new cell is run

#import helper 

## Import Data

In [4]:
FULL_FILE = 'data/brickset_features.csv'

In [5]:
full_df = pd.read_csv(FULL_FILE)
full_df.head()
full_df.info()

,set_no,name,theme_group,theme,subtheme,year,piece_cnt,minifig_cnt,price_store,price_new,...,rating_value,rating_votes,number_parts,total_parts,number_colors,1st_color,2nd_color,popularity,scolor_1st,scolor_2nd
0,1591-1,Danone Delivery Truck,Modern day,Town,Special,1980.0,40.0,0.0,NaN,129.0,...,NaN,NaN,19,40,4,White,Blue,NaN,White,Blue
1,1592-1,Town Square - Castle Scene,Modern day,Town,Classic,1980.0,471.0,11.0,NaN,1168.0,...,4.8,4.0,133,495,10,Black,Yellow,19.2,Black,Yellow
2,1651-2,Maersk Line Container Lorry,Modern day,Town,Special,1980.0,305.0,1.0,NaN,NaN,...,5.0,1.0,66,305,9,Maersk Blue,Light Gray,5.0,Blue,Gray
3,6305-1,Trees and Flowers,Modern day,Town,Accessories,1980.0,12.0,0.0,3.75,28.0,...,4.3,4.0,6,24,4,Green,Red,17.2,Green,Red
4,6306-1,Road Signs,Modern day,Town,Accessories,1980.0,12.0,0.0,2.50,49.0,...,4.0,3.0,10,12,1,White,White,12.0,White,White


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3213 entries, 0 to 3212
Data columns (total 21 columns):
set_no           3213 non-null object
name             3213 non-null object
theme_group      3213 non-null object
theme            3213 non-null object
subtheme         2667 non-null object
year             3213 non-null float64
piece_cnt        3212 non-null float64
minifig_cnt      3213 non-null float64
price_store      2762 non-null float64
price_new        3098 non-null float64
price_used       3140 non-null float64
rating_value     3001 non-null float64
rating_votes     3001 non-null float64
number_parts     3213 non-null int64
total_parts      3213 non-null int64
number_colors    3213 non-null int64
1st_color        3213 non-null object
2nd_color        3213 non-null object
popularity       3001 non-null float64
scolor_1st       3213 non-null object
scolor_2nd       3213 non-null object
dtypes: float64(9), int64(3), object(9)
memory usage: 527.2+ KB


## Split Out Test

In [6]:
X = full_df['price_store']
features_ls = ['', '', '', '', '', '', '']
y = full_df[''

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, random_state=1, stratify=y, test_size=0.3)

NameError: name 'train_test_split' is not defined